In [ ]:
%pip install tensorflow_io
%pip install -q kaggle
%pip install resampy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir -p ~/.kaggle
!cp -p drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!wget https://raw.githubusercontent.com/jackgle/YAMNet-transfer-learning/main/datagen_yamnet.py --quiet

In [ ]:
!kaggle datasets download -d soumendraprasad/musical-instruments-sound-dataset
!unzip -xq musical-instruments-sound-dataset.zip -d ./dataset 

In [ ]:
!cp /content/drive/MyDrive/yamnet.zip ./yamnet.zip
!unzip -xq yamnet.zip

In [2]:
import os
# os.sys.path.insert(0, yamnet_path)
from params import Params as params_ds
import features as features_lib
import shutil
import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import yamnet
from datagen_yamnet import DataGenerator, get_files_and_labels
import keras
from keras import Sequential
from keras.layers import Dense,Dropout
import tensorflow_io as tfio

In [3]:
# df=pd.read_csv("dataset/Metadata_Train.csv")
# df2=pd.read_csv("dataset/Metadata_Test.csv")
# classes=list(df['Class'].unique())
classes=list(os.listdir("dataset_emb/train"))
classes

['Sound_Drum', 'Sound_Guitar', 'Sound_Piano', 'Sound_Violin']

In [4]:
def to_dataset():
  dataset_p="/content/dataset_/"
  os.makedirs(dataset_p,exist_ok=True)
  shutil.rmtree(dataset_p)
  def fn(base_p,dst_p,df_):
    
    for i in classes:
      os.makedirs(dst_p+i,exist_ok=True)
    for fn,c in tqdm(df_.values):
      if c=='Sound_Guiatr':c='Sound_Guitar'
      # shutil.copy(base_p+fn,dst_p+c+)
      wav_data, sr = sf.read(base_p+fn, dtype=np.int16)
      assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
      waveform = wav_data / 32768.0
      
      if len(waveform.shape) > 1:
          waveform = waveform[:,0] # get left channel if multi-channel
      if sr != params_ds.sample_rate:
          waveform = resampy.resample(waveform, sr, params_ds.sample_rate)
          
      waveform = np.reshape(waveform, [1, -1]).astype(np.float32)
      _, patches = features_lib.waveform_to_log_mel_spectrogram_patches(tf.squeeze(waveform, axis=0), params_ds)
      np.save(dst_p+c+'/'+fn+'.npy', patches)
    
  #train
  train_base='/content/dataset/Train_submission/Train_submission/'
  train_dst=dataset_p+'train/'
  fn(train_base,train_dst,df)
  #test
  test_base='/content/dataset/Test_submission/Test_submission/'
  test_dst=dataset_p+"test/"
  fn(test_base,test_dst,df2)


In [ ]:
to_dataset()

In [ ]:
!cp -rf drive/MyDrive/dataset_/ dataset_/ 

In [ ]:
!cp -rf drive/MyDrive/dataset_emb/ dataset_emb/ 

In [3]:
!wget https://storage.googleapis.com/audioset/yamnet.h5 -q
weights_path='./yamnet.h5'

In [4]:
from keras import backend as K
K.clear_session()

In [5]:
import params
params = params.Params()   
# del yamnet_model   
weights_path='./yamnet.h5'                
yamnet_model = yamnet.yamnet_frames_model(params)
yamnet_model.load_weights(weights_path)

In [ ]:
def make_embedings():
  dataset_p="/content/dataset_emb/"
  os.makedirs(dataset_p,exist_ok=True)
  shutil.rmtree(dataset_p)
  def covert_to_int16(filename):
      wav_data, sr = sf.read(filename, dtype=np.int16)
      sf.write(filename,wav_data,sr)
  def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    covert_to_int16(filename)
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

  def fn(base_p,dst_p,df_):
    for i in classes:
      os.makedirs(dst_p+i,exist_ok=True)
    for fn,c in tqdm(df_.values):
      if c=='Sound_Guiatr':c='Sound_Guitar'
      # shutil.copy(base_p+fn,dst_p+c+)
      wav_data=load_wav_16k_mono(base_p+fn)
      predictions, embeddings, log_mel_spectrogram = yamnet_model(wav_data)
      np.save(dst_p+c+'/emb_'+fn+'.npy', embeddings)

  
  train_base='/content/dataset/Train_submission/Train_submission/'
  train_dst=dataset_p+'train/'
  fn(train_base,train_dst,df)
  #test
  test_base='/content/dataset/Test_submission/Test_submission/'
  test_dst=dataset_p+"test/"
  fn(test_base,test_dst,df2)

In [ ]:
make_embedings()

In [6]:

def x():

  def fn(path):
    ls_=[]
    for c in os.listdir(path):
      for name in os.listdir(f'{path}/{c}'):
        ls_.append((f'{path}{c}/{name}',c))
    return ls_
  p="dataset_emb/train/"
  train_df=pd.DataFrame(fn(p))
  train_df.to_csv("dataset_emb/train_files.csv",index=False)

  p="dataset_emb/test/"
  test_df=pd.DataFrame(fn(p))
  test_df.to_csv("dataset_emb/test_files.csv",index=False)

x()

In [7]:
base_model = tf.keras.models.Model(yamnet_model.get_layer('reshape').input,
                                   yamnet_model.get_layer('global_average_pooling2d').output,
                                   name='yamnet_base')

base_model.summary()

Model: "yamnet_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 64)]          0         
                                                                 
 reshape (Reshape)           (None, 96, 64, 1)         0         
                                                                 
 layer1/conv (Conv2D)        (None, 48, 32, 32)        288       
                                                                 
 layer1/conv/bn (BatchNormal  (None, 48, 32, 32)       96        
 ization)                                                        
                                                                 
 layer1/relu (ReLU)          (None, 48, 32, 32)        0         
                                                                 
 layer2/depthwise_conv (Dept  (None, 48, 32, 32)       288       
 hwiseConv2D)                                          

In [8]:
input_shape = (int(params.patch_window_seconds/params.stft_hop_seconds),
               params.mel_bands)

def get_model():
    
    inputs = tf.keras.layers.Input(input_shape)
    
    # call the base model
    # set batch norm layers to inference mode
    # https://keras.io/guides/transfer_learning/
    x = base_model(inputs, training=False)
    
    # define model top


    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x) 

    # x = tf.keras.layers.Dense(512,activation='relu')(x)
    # x = tf.keras.layers.Dropout(0.5)(x)

    outputs = tf.keras.layers.Dense(len(classes), activation='softmax')(x)
    
    # create model
    model = tf.keras.models.Model(inputs, outputs, name='custom_yamnet')
    
    return model

model = get_model()
model.summary()

Model: "custom_yamnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 64)]          0         
                                                                 
 yamnet_base (Functional)    (None, 1024)              3217344   
                                                                 
 dense_1 (Dense)             (None, 128)               131200    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 516       
                                                                 
Total params: 3,349,060
Trainable params: 3,327,172
Non-trainable params: 21,888
_________________________________________________________________


In [11]:
train_dir = './dataset_emb/train/'
files_train, files_val, labels = get_files_and_labels(train_dir, 
                                                    typ='npy',
                                                    train_split=0.8)

In [12]:
batch_size = 3

train_generator = DataGenerator(files_train,
                                labels,
                                n_classes=len(classes),
                                batch_size=batch_size)
validation_generator = DataGenerator(files_val,
                                    labels,
                                    n_classes=len(classes),
                                    batch_size=batch_size)

model_out="models/model_y"
# define training callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_out+'.h5',
                                                 monitor='val_loss', 
                                                 verbose=1,
                                                 save_best_only=True, 
                                                 mode='auto')
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

checkpoint = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min', min_lr=0.0001)
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)
csv_logger = keras.callbacks.CSVLogger('training.log', separator=',', append=False)
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.1
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

In [13]:
num_classes=len(classes)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model)

In [ ]:
model.get_layer('yamnet_base').trainable = False

# compile model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer,metrics=[keras.metrics.categorical_accuracy,])

model_history = model.fit(train_generator,
                            steps_per_epoch = len(train_generator),
                            epochs = 12,
                            validation_data = validation_generator,
                            validation_steps = len(validation_generator),
                            verbose = 1,
                            callbacks=[checkpoint,
early_stop,

checkpoint,
reduce_lr,
tensorboard,
csv_logger,
lr_scheduler])

In [ ]:
!cp -rfp ./dataset_emb drive/MyDrive/dataset_emb

# training of saved embeddings

In [4]:
train_df_emb=pd.read_csv('dataset_emb/train_files.csv')
test_df_emb=pd.read_csv('dataset_emb/test_files.csv')

In [5]:
train_df_emb = train_df_emb.sample(frac=1)
test_df_emb = test_df_emb.sample(frac=1)

In [6]:
train_df_emb.shape

(2629, 2)

In [21]:

my_model=Sequential([
    # Dense(1024,activation='relu',),
    keras.layers.Dense(128, input_shape=(1024,),activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),
    Dense(4,activation='softmax'),
])
from keras.losses import categorical_crossentropy
my_model.compile(loss=categorical_crossentropy,metrics=['categorical_accuracy'])

In [29]:
my_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               131200    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 4)                 516       
                                                                 
Total params: 131,716
Trainable params: 131,716
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.utils import to_categorical
def to_ds(t):
  # print(t)
  f_n,c=t
  emb=np.load(f_n).astype("float32")
  one_c=to_categorical(classes.index(c),len(classes))
  num_emb=emb.shape[0]
  return emb,np.tile(one_c, (num_emb, 1))
   
Xy=train_df_emb.apply(to_ds,axis=1)
Xy_t=test_df_emb.apply(to_ds,axis=1)

In [12]:
X=[]
y=[]
for i,j in Xy:
  for x_ in i:
    X.append(x_)
  for y_ in j:
    y.append(y_)
X=np.array(X,dtype='float32')
y=np.array(y,dtype='float32')

In [13]:
X_t=[]
y_t=[]
for i,j in Xy_t:
  for x_ in i:
    X_t.append(x_)
  for y_ in j:
    y_t.append(y_)
X_t=np.array(X_t,dtype='float32')
y_t=np.array(y_t,dtype='float32')

In [14]:
X.shape,y.shape

((96688, 1024), (96688, 4))

In [15]:
X_t.shape,y_t.shape

((3965, 1024), (3965, 4))

In [16]:
data_size=X.shape[0]
split_ind=int(data_size*.85)
X_train,y_train=X[:split_ind],y[:split_ind]
X_val,y_val=X[split_ind:],y[split_ind:]

In [19]:
model_out="models/model_y_emb"

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, verbose=1, )
checkpoint = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=0.00001)
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)
csv_logger = keras.callbacks.CSVLogger('training.log', separator=',', append=False)


In [ ]:
hist=my_model.fit(X,y,batch_size=32,epochs=20,validation_data=(X_val,y_val),shuffle=True,callbacks=[checkpoint,
early_stop,
checkpoint,
reduce_lr,
tensorboard,
csv_logger,
],
)

In [27]:
res=my_model.evaluate(X_t,y_t) 
res

124/124 [==============================] - 1s 4ms/step - loss: 1.2753 - categorical_accuracy: 0.7269


[1.2753185033798218, 0.7268600463867188]

In [26]:
X.var()

0.4504392

In [28]:
my_model.save('best_top/',overwrite=True)

INFO:tensorflow:Assets written to: best_top/assets


In [ ]:
def eval():
    eval = my_model.evaluate(X_t,y_t,verbose=-1)
    pred = my_model.predict(X_t,verbose=-1)
    pred_mean = np.mean(pred,axis=0,dtype='float32')
    pred_dict={j:float(i) for i,j in zip(pred_mean,classes_key)}
    # exp=np.exp(pred_mean)
    # pred_softmax=exp/np.sum(exp)
    rows.append([f_n,c,classes_key[pred_mean.argmax()],eval[1],eval[0],*pred_mean])
    res.append((f_n,c,eval,pred_dict))
    df=pd.DataFrame(rows ,columns=['filename','actual_class','predicted_class','eval_scroe',"eval_loss",'Sound_Drum','Sound_Guitar',"Sound_Piano","Sound_Violin"])
    df.to_csv("res.csv",index=False)

In [31]:
eval = my_model.evaluate(X_t,y_t,verbose=-1)
pred = my_model.predict(X_t,verbose=-1)

In [50]:
pred_df=pd.DataFrame(pred, columns =classes)

In [51]:
pred_df['actual_class']=y_t.argmax(axis=1)

In [52]:
pred_df.to_csv("pred.csv",index=False)

In [60]:
from sklearn.metrics import classification_report
import numpy as np

Y_test = np.argmax(y_t, axis=1) # Convert one-hot to index
y_pred = np.argmax(pred,axis=1)
print(classification_report(Y_test, y_pred,target_names=classes))
print(classification_report(Y_test, y_pred,output_dict=True,target_names=classes))

              precision    recall  f1-score   support

  Sound_Drum       0.80      0.79      0.80       900
Sound_Guitar       0.94      0.67      0.78      1619
 Sound_Piano       0.68      0.93      0.78      1139
Sound_Violin       0.09      0.11      0.10       307

    accuracy                           0.73      3965
   macro avg       0.63      0.62      0.61      3965
weighted avg       0.77      0.73      0.73      3965

{'Sound_Drum': {'precision': 0.801354401805869, 'recall': 0.7888888888888889, 'f1-score': 0.7950727883538634, 'support': 900}, 'Sound_Guitar': {'precision': 0.9408695652173913, 'recall': 0.6683137739345275, 'f1-score': 0.7815095702419645, 'support': 1619}, 'Sound_Piano': {'precision': 0.6784338896020539, 'recall': 0.9280070237050044, 'f1-score': 0.7838338895068594, 'support': 1139}, 'Sound_Violin': {'precision': 0.0889487870619946, 'recall': 0.10749185667752444, 'f1-score': 0.09734513274336284, 'support': 307}, 'accuracy': 0.7268600252206809, 'macro avg': {'p

In [61]:
l=[]
for i,j in zip(y_pred,Y_test):
    if (j==3):#Sound_Violin
        l.append(classes[i])
from collections import Counter
Counter(l)


Counter({'Sound_Piano': 195,
         'Sound_Guitar': 46,
         'Sound_Violin': 33,
         'Sound_Drum': 33})

In [ ]:
{#violin miss prediction of classes 
#these are the prediction for violin class predictions
'Sound_Piano': 195,
'Sound_Guitar': 46,
'Sound_Violin': 33,
'Sound_Drum': 33
}

In [63]:
X.shape

(96688, 1024)

In [64]:
pd.DataFrame({'Sound_Drum': {'precision': 0.801354401805869, 'recall': 0.7888888888888889, 'f1-score': 0.7950727883538634, 'support': 900}, 'Sound_Guitar': {'precision': 0.9408695652173913, 'recall': 0.6683137739345275, 'f1-score': 0.7815095702419645, 'support': 1619}, 'Sound_Piano': {'precision': 0.6784338896020539, 'recall': 0.9280070237050044, 'f1-score': 0.7838338895068594, 'support': 1139}, 'Sound_Violin': {'precision': 0.0889487870619946, 'recall': 0.10749185667752444, 'f1-score': 0.09734513274336284, 'support': 307}, 'accuracy': 0.7268600252206809, 'macro avg': {'precision': 0.6274016609218271, 'recall': 0.6231753858014862, 'f1-score': 0.6144403452115125, 'support': 3965}, 'weighted avg': {'precision': 0.7678512649677202, 'recall': 0.7268600252206809, 'f1-score': 0.732282789316707, 'support': 3965}}
)

,Sound_Drum,Sound_Guitar,Sound_Piano,Sound_Violin,accuracy,macro avg,weighted avg
precision,0.801354,0.940870,0.678434,0.088949,0.72686,0.627402,0.767851
recall,0.788889,0.668314,0.928007,0.107492,0.72686,0.623175,0.726860
f1-score,0.795073,0.781510,0.783834,0.097345,0.72686,0.614440,0.732283
support,900.000000,1619.000000,1139.000000,307.000000,0.72686,3965.000000,3965.000000


In [ ]:
X_t